#### Assignment No.4
Perform text classification using Naive Bayes/Logistic
Regression/ Support Vector Machines /Decision Tree
Classifier. Use dataset "Economic news article tone and
relevance" news articles, which were tagged as relevant or not
relevant to the US Economy. Keep the required columns and
save it in a dataframe. Explore the process of training and
testing text classifier for this dataset and analyse the
performance.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gadad\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gadad\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\gadad\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gadad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\gadad\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### 1.Import Dataset

In [3]:
df=pd.read_csv('Economic_News.csv',encoding=('ISO-8859-1'),index_col=0)
df.head()

,_golden,_unit_state,_trusted_judgments,_last_judgment_at,positivity,positivity:confidence,relevance,relevance:confidence,articleid,date,headline,positivity_gold,relevance_gold,text
_unit_id,,,,,,,,,,,,,,
842613455,False,finalized,3,12/5/15 17:48,3.0,0.6400,yes,0.640,wsj_398217788,8/14/91,Yields on CDs Fell in the Latest Week,NaN,NaN,NEW YORK -- Yields on most certificates of dep...
842613456,False,finalized,3,12/5/15 16:54,NaN,NaN,no,1.000,wsj_399019502,8/21/07,The Morning Brief: White House Seeks to Limit ...,NaN,NaN,The Wall Street Journal Online</br></br>The Mo...
842613457,False,finalized,3,12/5/15 1:59,NaN,NaN,no,1.000,wsj_398284048,11/14/91,Banking Bill Negotiators Set Compromise --- Pl...,NaN,NaN,WASHINGTON -- In an effort to achieve banking ...
842613458,False,finalized,3,12/5/15 2:19,NaN,0.0000,no,0.675,wsj_397959018,6/16/86,Manager's Journal: Sniffing Out Drug Abusers I...,NaN,NaN,The statistics on the enormous costs of employ...
842613459,False,finalized,3,12/5/15 17:48,3.0,0.3257,yes,0.640,wsj_398838054,10/4/02,Currency Trading: Dollar Remains in Tight Rang...,NaN,NaN,NEW YORK -- Indecision marked the dollar's ton...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8000 entries, 842613455 to 830985636
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   _golden                8000 non-null   bool   
 1   _unit_state            8000 non-null   object 
 2   _trusted_judgments     8000 non-null   int64  
 3   _last_judgment_at      8000 non-null   object 
 4   positivity             1420 non-null   float64
 5   positivity:confidence  3775 non-null   float64
 6   relevance              8000 non-null   object 
 7   relevance:confidence   8000 non-null   float64
 8   articleid              8000 non-null   object 
 9   date                   8000 non-null   object 
 10  headline               8000 non-null   object 
 11  positivity_gold        0 non-null      float64
 12  relevance_gold         0 non-null      float64
 13  text                   8000 non-null   object 
dtypes: bool(1), float64(5), int64(1), object(7)

In [5]:
display(df.shape) # Number of rows (instances) and columns in the dataset
df["relevance"].value_counts()

(8000, 14)

no          6571
yes         1420
not sure       9
Name: relevance, dtype: int64

In [6]:
df["relevance"].value_counts()/df.shape[0]# Class distribution in the dataset

no          0.821375
yes         0.177500
not sure    0.001125
Name: relevance, dtype: float64

There is an imbalance in the data with not relevant being 82% in the dataset. That is, most of the articles are not relevant to US Economy, which makes sense in a real-world scenario, as news articles discuss various topics. We should keep this class imbalance mind when interpreting the classifier performance later. Let us first convert the class labels into binary outcome variables for convenience. 1 for Yes (relevant), and 0 for No (not relevant), and ignore "Not sure".

### 2.Label Encodeing

In [19]:
# Define a mapping for label encoding
label_mapping = {'no': 0, 'yes': 1}

# Apply label encoding to the 'label' column
df['label_num'] = df['relevance'].map(label_mapping)
df = df[["text","label_num"]] # Let us take only the two columns we need.
df.shape

(8000, 2)

In [20]:
# # convert label to a numerical variable
# df = df[df.relevance != "not sure"] # removing the data where we don't want relevance="not sure".
# df.shape
# df['relevance'] = df.relevance.map({'yes':1, 'no':0}) # relevant is 1, not-relevant is 0. 
# df = df[["text","relevance"]] # Let us take only the two columns we need.
# df.shape

In [21]:
df.head()

,text,label_num
_unit_id,,
842613455,new york yields certificates deposit offered m...,1.0
842613456,wall street journal onlinebrbrthe morning brie...,0.0
842613457,washington effort achieve banking reform senat...,0.0
842613458,statistics enormous costs employee drug abuse ...,0.0
842613459,new york indecision marked dollars tone trader...,1.0


### 3.Define a Preprocessing Function:

In [22]:
def preprocess_text(Message):
    # Convert text to lowercase
    Message = Message.lower()
    
    # Remove special characters, numbers, and punctuation
    Message = re.sub(r'[^a-z\s]', '', Message)
    
    # Tokenize the text
    tokens =nltk.word_tokenize(Message)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Join tokens back into a string
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text
df['text'] = df['text'].apply(preprocess_text)


In [27]:
df = df.dropna()
x=df['text']
y=df['label_num']

In [28]:
print(x)

_unit_id
842613455    new york yields certificates deposit offered m...
842613456    wall street journal onlinebrbrthe morning brie...
842613457    washington effort achieve banking reform senat...
842613458    statistics enormous costs employee drug abuse ...
842613459    new york indecision marked dollars tone trader...
                                   ...                        
830985632    secretary commerce charles w sawyer said yeste...
830985633    us stocks inched last week overcoming concern ...
830985634    ben bernanke cleared key hurdle thursday confi...
830985635    white houses push contract many federal functi...
830985636    new york april automobile stocks put best show...
Name: text, Length: 7991, dtype: object


### 4.Feature Extraction and Model Training:

In [29]:
# Assuming you have the features (X) and labels (y) from your dataset
# Apply TF-IDF Vectorization

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf_vectorizer.fit_transform([preprocess_text(Message) for Message in x])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Initialize and train the Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)


MultinomialNB()

### 5. Model Evaluation

In [30]:
# Predict and evaluate the model
y_pred = nb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{classification_rep}")


Accuracy: 0.8136335209505942
Classification Report:
              precision    recall  f1-score   support

         0.0       0.82      0.99      0.90      1302
         1.0       0.48      0.05      0.09       297

    accuracy                           0.81      1599
   macro avg       0.65      0.52      0.49      1599
weighted avg       0.76      0.81      0.75      1599



### Taking User Input

In [33]:
# Assuming `new_message` is the new email you want to classify
new_message = input("Your message here: ") # Replace with your actual message
preprocessed_message = preprocess_text(new_message)
tfidf_vectorized_message = tfidf_vectorizer.transform([preprocessed_message])

prediction = nb_classifier.predict(tfidf_vectorized_message)
print(f"Predicted Class: {'yes' if prediction == 1 else 'No'}")


Your message here: new york indecision marked dollars tone trader
Predicted Class: No
